In [12]:
import pandas as pd
from googleapiclient.discovery import build
from IPython.display import JSON
import json
import regex as re

### Grab API Key

In [13]:
from api import api_key

In [14]:
channel_ids = ["UCX6b17PVsYBQ0ip5gyeme-Q"]

In [15]:
def get_channel_stats(channel_data):
    channels = []
    for item in channel_data["items"]:
        dict = {}
        dict["channelName"] = item["snippet"]["title"]
        dict["publishedDate"] = item["snippet"]["publishedAt"]
        dict["subscribers"] = item["statistics"]["subscriberCount"]
        dict["views"] = item["statistics"]["viewCount"]
        dict["totalVideos"] = item["statistics"]["videoCount"]
        dict["playlistId"] = item["contentDetails"]["relatedPlaylists"]['uploads']

        channels.append(dict)
    return channels


In [16]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id = ",".join(channel_ids)
)
response = request.execute()


In [17]:

print(json.dumps(response, indent=2))

{
  "kind": "youtube#channelListResponse",
  "etag": "ehByGjtoIy0PktRTfldf1V6Y6Ls",
  "pageInfo": {
    "totalResults": 1,
    "resultsPerPage": 5
  },
  "items": [
    {
      "kind": "youtube#channel",
      "etag": "fBpaDocjOdf1gaDwns4Qu-jysuU",
      "id": "UCX6b17PVsYBQ0ip5gyeme-Q",
      "snippet": {
        "title": "CrashCourse",
        "description": "At Crash Course, we believe that high-quality educational videos should be available to everyone for free! \n\nSubscribe for weekly videos from our current courses! Right now, we're producing Climate & Energy. The Crash Course team has produced more than 45 courses on a wide variety of subjects, including organic chemistry, literature, world history, biology, philosophy, theater, ecology, and many more!  We also recently teamed up with Arizona State University to bring you more courses on the Study Hall channel.\n\nHelp support Crash Course at Patreon.com/CrashCourse.",
        "customUrl": "@crashcourse",
        "publishedAt":

In [18]:
channel_stats = get_channel_stats(response)

In [19]:
channel_stats[0]

{'channelName': 'CrashCourse',
 'publishedDate': '2006-05-20T02:43:42Z',
 'subscribers': '14800000',
 'views': '1828997009',
 'totalVideos': '1475',
 'playlistId': 'UUX6b17PVsYBQ0ip5gyeme-Q'}

In [20]:
def get_video_ids(youtube, playlist_id):
    video_ids=[]

    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults = 50)

    response = request.execute()

    for item in response["items"]:
            video_ids.append(item["contentDetails"]["videoId"])

    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        )

        response = request.execute()

        for item in response["items"]:
            video_ids.append(item["contentDetails"]["videoId"])

        next_page_token = response.get("nextPageToken")

    return video_ids

In [21]:
video_ids = get_video_ids(youtube,channel_stats[0]["playlistId"])
len(video_ids)

1475

### Get video stats

In [22]:
def get_video_stats(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids), 50):

        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(video_ids[i:i+50])
        )
        response = request.execute()

        

        for video in response["items"]:
            stats_to_keep = {"snippet": ["channelTitle", "title", "description", "tags", "publishedAt"],
                            "statistics": ["viewCount", "likeCount", "favouriteCount", "commentCount"],
                            "contentDetails": ["duration", "definition", "caption"]}
            video_info = {}
            video_info["video_id"] = video["id"]

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None
            
            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [23]:
cc_df = get_video_stats(youtube, video_ids)
cc_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,WzOrF5W4l3Q,CrashCourse,Photosynthesis and Cellular Respiration: Crash...,"Plants and trees may seem pretty passive, but ...","[vlogbrothers, Crash Course, crashcourse, educ...",2023-06-22T16:00:21Z,13760,835,None,51,PT13M,hd,true
1,pqosCR6J2qo,CrashCourse,What Do These Creepy Plant Mouths Do? (Plant T...,"Plants—they’re just like us! Well, not exactly...","[vlogbrothers, Crash Course, crashcourse, educ...",2023-06-15T16:00:38Z,32496,1570,None,52,PT12M44S,hd,true
2,xOLcZMw0hd4,CrashCourse,The Scientific Method: Crash Course Biology #2,Science offers a way of discovering and unders...,"[vlogbrothers, Crash Course, crashcourse, educ...",2023-06-13T16:00:44Z,30368,1519,None,87,PT15M9S,hd,true
3,tZE_fQFK8EY,CrashCourse,Introduction to Biology: Crash Course Biology #1,Biology is the study of life—a four-letter wor...,"[vlogbrothers, Crash Course, crashcourse, educ...",2023-06-06T16:00:39Z,70777,3775,None,266,PT13M27S,hd,true
4,y9BLCfcUcFg,CrashCourse,Plant Cells & Hormones: Crash Course Botany #3,"At first glance, plant and animal cells have a...","[vlogbrothers, Crash Course, crashcourse, educ...",2023-06-01T16:03:13Z,56483,3462,None,135,PT12M58S,hd,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,HVT3Y3_gHGg,CrashCourse,Water - Liquid Awesome: Crash Course Biology #2,Hank teaches us why water is one of the most f...,"[water, hydrogen, oxygen, molecule, covalent b...",2012-02-06T22:10:10Z,4840139,51024,None,3825,PT11M17S,hd,true
1471,n7ndRwqJYDM,CrashCourse,Indus Valley Civilization: Crash Course World ...,In which John Green teaches you about the Indu...,"[John Green, Crashcourse, Documentary, Valley,...",2012-02-02T20:18:11Z,7917297,85507,None,8047,PT9M35S,hd,true
1472,QnQe0xW_JY4,CrashCourse,Carbon... SO SIMPLE: Crash Course Biology #1,Check out our new-and-improved Crash Course Bi...,"[biology, crashcourse, gilbert lewis, carbon, ...",2012-01-30T18:53:06Z,7827237,78209,None,7809,PT11M57S,hd,true
1473,Yocja_N5s1I,CrashCourse,The Agricultural Revolution: Crash Course Worl...,In which John Green investigates the dawn of h...,"[John Green, history, agriculture, ancient, pr...",2012-01-26T20:12:01Z,15378590,152936,None,12247,PT11M11S,hd,true


## Load raw data to CSV 

In [37]:
cc_df.to_csv("data/crashcourse_raw.csv")

## Data Pre-processing 

In [101]:
cc_df_copy = cc_df.copy()

### Clean course information

In [102]:
# Fix CrashCourse
def fix_cc(title):
    if re.search(r"CrashCourse",title):
        return title.replace("CrashCourse","Crash Course")
    return title

In [103]:
cc_df_copy["title"] = cc_df_copy["title"].apply(lambda x: fix_cc(x))

In [104]:
# Fix "Ai" versus "AI"
def fix_AI(title):
    if re.search(r"Ai",title):
        return title.replace("Ai","AI")
    return title

In [105]:
cc_df_copy["title"] = cc_df_copy["title"].apply(lambda x: fix_AI(x))

### Get course information

In [46]:
# Function to parse out information if it is a regular crashcourse video (og) 
def regular_title_info(id, x):
    # initialize dictionary
    dict={}

    # Split the title by the words "Crash Course"
    title_parts = x.split("Crash Course")

    # Grab the episode title and get rid of the ":"
    episode = title_parts[0].replace(":","")
    dict["episode_title"] = episode

    # Split the course and the episode number along the "#"
    try:
        course_info = title_parts[1].split("#")
        dict["course"] = course_info[0]    
        dict["episode_num"] = course_info[1]
    except:
        dict["course"] = title_parts
        dict["episode_num"] = None

    # Grab the length of the raw title of the episode 
    dict["episode_title_len"] = len(episode)

    # Include video_id
    dict["video_id"] = id

    # Include Arizona teamup
    dict["with_Arizona"] = False

    # include course type
    dict["video_type"] = "regular course"

    return dict

In [30]:
# Function to grab video information if type is preview
def preview_title_info(id, x):
    # initialize dictionary
    dict = {}

    dict["episode_title"] = x.replace("Preview", "").replace(":","")
    dict["course"] = "Preview"
    dict["episode_num"] = None
    dict["episode_title_len"] = len((x.replace("Preview", "").replace(":","")))
    dict["video_id"] = id
    dict["with_Arizona"] = False
    dict["video_type"] = "preview"

    return dict

In [31]:
# Function to grab video information if type "How to College":
def college_title_info(id,x):
    # initialize dictionary
    dict = {}
    episode_title =  x.replace("How to College", "").replace("Crash Course","")

    dict["episode_title"] = episode_title
    dict["course"] = "How to College"
    dict["episode_num"] = None
    dict["episode_title_len"] = len(episode_title)
    dict["video_id"] = id
    dict["with_Arizona"] = False
    dict["video_type"] = "How to College"

    return dict

In [98]:
# Function to grab video information if type is outtakes
def outtakes_title_info(id, x):
    # initialize dictionary
    dict = {}

    dict["episode_title"] = x.replace("Crash Course", "").replace(":","")
    dict["course"] = "outtakes"
    dict["episode_num"] = None
    dict["episode_title_len"] = len((x.replace("Crash Course", "")))
    dict["video_id"] = id
    dict["with_Arizona"] = False
    dict["video_type"] = "outtakes"

    return dict

In [33]:
# Function to grab any exceptions to the titles
def other_title_info(id, x):
    dict = {}

    dict["episode_title"] = x
    dict["course"] = "unknown"
    dict["episode_num"] = None
    dict["episode_title_len"] = len(x)
    dict["video_id"] = id
    dict["with_Arizona"] = False
    dict["video_type"] = "unknown"

    return dict

In [39]:
# Function to grab any exceptions to the titles
def office_title_info(id, x):
    dict = {}

    episode =  x.replace("Crash Course","").replace("Office Hourse","").replace(":","")
    dict["episode_title"] = episode
    dict["course"] = "Office Hours"
    dict["episode_num"] = None
    dict["episode_title_len"] = len(episode)
    dict["video_id"] = id
    dict["with_Arizona"] = False
    dict["video_type"] = "office hours"

    return dict

In [82]:
# Function to grab the Crash Course Literature
def lit_title_info(id, x):
    dict = {}

    # Inconsistent Naming even within the same season (some contain ":" and other's contains "-")
    if re.search(":",x):
        episode = x.split(":")
    elif re.search("-",x):
        episode = x.split("-")
    else: 
        episode = x.split("!")
        episode[0] = "Disease!"
     
    episode_num = episode[1].replace("Crash Course Literature","").replace("Crash Course World History","").replace("Crash Course Big History","")
    dict["episode_title"] = episode[0]
    
    # Figure out which course it is a subsequent season of
    if re.search(r"Crash Course Literarture",x):
        dict["course"] = "Literature"
    elif re.search(r"Crash Course World History",x):
        dict["course"] = "World History"
    else:
        dict["course"] = "Big History"

    dict["episode_num"] = episode_num
    dict["episode_title_len"] = len(episode[0])
    dict["video_id"] = id
    dict["with_Arizona"] = False
    dict["video_type"] = "regular course"

    return dict

In [100]:
def grab_course(video_id, title):
    title_info = []

    # Loop through the titles and make a dicitonary of each one
    for index, x in enumerate(title):

        if re.search("Preview",x): #FINISHED
            dict = preview_title_info(video_id[index], x)

        elif re.search("Outtakes",x):
            dict = outtakes_title_info(video_id[index],x)
            
        elif re.search("How to College",x):
            dict = college_title_info(video_id[index], x)

        elif re.search("Office Hours",x):
            dict = office_title_info(video_id[index],x)

        elif re.search("#",x): #FINISHED
            dict = regular_title_info(video_id[index], x)

        elif re.search("Crash Course Literature",x) or re.search("Crash Course World History",x) or re.search("Crash Course Big History",x):
            dict = lit_title_info(video_id[index],x)
            
        else:
            dict = other_title_info(video_id[index], x)
            

        # Add the dictionary to the list
        title_info.append(dict)

    # Return information as a dataframe
    return pd.DataFrame(title_info)

In [106]:
# Apply grab_course_info function on the title column of the dataframe
title_df = grab_course(cc_df_copy["video_id"], cc_df_copy["title"])
title_df

,episode_title,course,episode_num,episode_title_len,video_id,with_Arizona,video_type
0,Photosynthesis and Cellular Respiration,Botany,5,40,WzOrF5W4l3Q,False,regular course
1,What Do These Creepy Plant Mouths Do? (Plant Tissues),Botany,4,54,pqosCR6J2qo,False,regular course
2,The Scientific Method,Biology,2,22,xOLcZMw0hd4,False,regular course
3,Introduction to Biology,Biology,1,24,tZE_fQFK8EY,False,regular course
4,Plant Cells & Hormones,Botany,3,23,y9BLCfcUcFg,False,regular course
...,...,...,...,...,...,...,...
1470,Water - Liquid Awesome,Biology,2,23,HVT3Y3_gHGg,False,regular course
1471,Indus Valley Civilization,World History,2,26,n7ndRwqJYDM,False,regular course
1472,Carbon... SO SIMPLE,Biology,1,20,QnQe0xW_JY4,False,regular course
1473,The Agricultural Revolution,World History,1,28,Yocja_N5s1I,False,regular course


### Check for what is in the unknowns

In [90]:
pd.set_option('display.max_colwidth', 1000)

In [91]:
title_df[title_df["course"]=="unknown"]

,episode_title,course,episode_num,episode_title_len,video_id,with_Arizona,video_type
6,NEW Crash Course Biology Series!,unknown,None,32,PWGBqskV1UQ,False,unknown
232,Covid-19 and Public Health: A Message from Crash Course,unknown,None,55,G4rcv3p7AYg,False,unknown
262,Announcing the Crash Course App!,unknown,None,32,JfMmzx67Krw,False,unknown
295,What's new with Crash Course,unknown,None,28,seLtYzXJ_YE,False,unknown
428,A History of Crash Course,unknown,None,25,1Q5cPfbmSD8,False,unknown
654,Changes to our Patreon,unknown,None,22,JeOimlgs5tQ,False,unknown
837,A Note on CC Human Geography,unknown,None,28,yvFStAP7Uko,False,unknown
960,Explore The Solar System: 360 Degree Interactive Tour!,unknown,None,54,0ytyMKa8aps,False,unknown
1068,Crash Course Economics Intro!,unknown,None,29,9I_-ADGrKQo,False,unknown
1126,Become a Crash Course Patron!,unknown,None,29,VO_1VbQADW8,False,unknown


### Concatenate the course information df with the rest of cc_df_copy

In [111]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
# Concatenate the course information (title_df) with the cc_df_copy
cc_bycourse_df = pd.merge(cc_df_copy, title_df, on="video_id")

# Drop the title column
cc_bycourse_df.drop(columns=["title"], inplace=True)
cc_bycourse_df.head(5)

### Check if amy columns have null values

In [112]:
cc_bycourse_df.isnull().any()

video_id             False
channelTitle         False
description          False
tags                  True
publishedAt          False
viewCount             True
likeCount             True
favouriteCount        True
commentCount          True
duration             False
definition           False
caption              False
episode_title        False
course               False
episode_num           True
episode_title_len    False
with_Arizona         False
video_type           False
dtype: bool

### Check Column Types

In [113]:
cc_bycourse_df.dtypes

video_id             object
channelTitle         object
description          object
tags                 object
publishedAt          object
viewCount            object
likeCount            object
favouriteCount       object
commentCount         object
duration             object
definition           object
caption              object
episode_title        object
course               object
episode_num          object
episode_title_len     int64
with_Arizona           bool
video_type           object
dtype: object

In [114]:
# Convert "viewCount", "likeCount", "favouriteCount", "commentCount" to numeric
numeric_cols = ["viewCount", "likeCount", "favouriteCount", "commentCount", "episode_num"]
cc_bycourse_df[numeric_cols] = cc_bycourse_df[numeric_cols].apply(pd.to_numeric, errors="coerce", axis=1)

In [115]:
cc_bycourse_df.dtypes

video_id              object
channelTitle          object
description           object
tags                  object
publishedAt           object
viewCount            float64
likeCount            float64
favouriteCount       float64
commentCount         float64
duration              object
definition            object
caption               object
episode_title         object
course                object
episode_num          float64
episode_title_len      int64
with_Arizona            bool
video_type            object
dtype: object

### Convert duration column to seconds

In [116]:
import isodate

In [117]:
cc_bycourse_df["duration"] = cc_bycourse_df["duration"].apply(lambda x: isodate.parse_duration(x))
cc_bycourse_df["duration"] = cc_bycourse_df["duration"].astype("timedelta64[s]")
cc_bycourse_df.head(5)

,video_id,channelTitle,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,episode_title,course,episode_num,episode_title_len,with_Arizona,video_type
0,WzOrF5W4l3Q,CrashCourse,"Plants and trees may seem pretty passive, but behind the scenes, their cells are working hard to...","[vlogbrothers, Crash Course, crashcourse, education, botany, photosynthesis, mitochondria, chlor...",2023-06-22T16:00:21Z,13760.0,835.0,NaN,51.0,780.0,hd,true,Photosynthesis and Cellular Respiration,Botany,5.0,40,False,regular course
1,pqosCR6J2qo,CrashCourse,"Plants—they’re just like us! Well, not exactly, but they do have skin and hair like us…even if t...","[vlogbrothers, Crash Course, crashcourse, education, botany, photosynthesis, plant fibers]",2023-06-15T16:00:38Z,32496.0,1570.0,NaN,52.0,764.0,hd,true,What Do These Creepy Plant Mouths Do? (Plant Tissues),Botany,4.0,54,False,regular course
2,xOLcZMw0hd4,CrashCourse,"Science offers a way of discovering and understanding the world around us, driven by questions a...","[vlogbrothers, Crash Course, crashcourse, education, Sammy Ramsey, Dr. Sammy, Dr. Bugs, biology,...",2023-06-13T16:00:44Z,30368.0,1519.0,NaN,87.0,909.0,hd,true,The Scientific Method,Biology,2.0,22,False,regular course
3,tZE_fQFK8EY,CrashCourse,Biology is the study of life—a four-letter word that connects you to 4 billion years worth of fa...,"[vlogbrothers, Crash Course, crashcourse, education, Sammy Ramsey, Dr. Sammy, Dr. Bugs, biology,...",2023-06-06T16:00:39Z,70777.0,3775.0,NaN,266.0,807.0,hd,true,Introduction to Biology,Biology,1.0,24,False,regular course
4,y9BLCfcUcFg,CrashCourse,"At first glance, plant and animal cells have a lot in common: they’re both highly organized, kee...","[vlogbrothers, Crash Course, crashcourse, education, crash course, botany, plants, agriculture, ...",2023-06-01T16:03:13Z,56483.0,3462.0,NaN,135.0,778.0,hd,true,Plant Cells & Hormones,Botany,3.0,23,False,regular course
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,HVT3Y3_gHGg,CrashCourse,Hank teaches us why water is one of the most fascinating and important substances in the univers...,"[water, hydrogen, oxygen, molecule, covalent bond, cohesion, adhesion, polarity, hydrogen bond, ...",2012-02-06T22:10:10Z,4840139.0,51024.0,NaN,3825.0,677.0,hd,true,Water - Liquid Awesome,Biology,2.0,23,False,regular course
1471,n7ndRwqJYDM,CrashCourse,"In which John Green teaches you about the Indus Valley Civilization, one of the largest of the a...","[John Green, Crashcourse, Documentary, Valley, Culture, History, vlogbrothers, India, Pakistan, ...",2012-02-02T20:18:11Z,7917297.0,85507.0,NaN,8047.0,575.0,hd,true,Indus Valley Civilization,World History,2.0,26,False,regular course
1472,QnQe0xW_JY4,CrashCourse,Check out our new-and-improved Crash Course Biology series here! https://youtube.com/playlist?li...,"[biology, crashcourse, gilbert lewis, carbon, hydrogen, electron, proton, covalent bonds, ion, o...",2012-01-30T18:53:06Z,7827237.0,78209.0,NaN,7809.0,717.0,hd,true,Carbon... SO SIMPLE,Biology,1.0,20,False,regular course
1473,Yocja_N5s1I,CrashCourse,In which John Green investigates the dawn of human civilization. John looks into how people gave...,"[John Green, history, agriculture, ancient, pre-history, prehistoric, farming, double, cheesebur...",2012-01-26T20:12:01Z,15378590.0,152936.0,NaN,12247.0,671.0,hd,true,The Agricultural Revolution,World History,1.0,28,False,regular course


### Grab the number of tags for each video

In [118]:
# If there are no tags, then put None
cc_bycourse_df["tag_count"] = cc_bycourse_df["tags"].apply(lambda x: 0 if x is None else len(x))
cc_bycourse_df.head(2)

,video_id,channelTitle,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,episode_title,course,episode_num,episode_title_len,with_Arizona,video_type,tag_count
0,WzOrF5W4l3Q,CrashCourse,"Plants and trees may seem pretty passive, but behind the scenes, their cells are working hard to...","[vlogbrothers, Crash Course, crashcourse, education, botany, photosynthesis, mitochondria, chlor...",2023-06-22T16:00:21Z,13760.0,835.0,NaN,51.0,780.0,hd,true,Photosynthesis and Cellular Respiration,Botany,5.0,40,False,regular course,15
1,pqosCR6J2qo,CrashCourse,"Plants—they’re just like us! Well, not exactly, but they do have skin and hair like us…even if t...","[vlogbrothers, Crash Course, crashcourse, education, botany, photosynthesis, plant fibers]",2023-06-15T16:00:38Z,32496.0,1570.0,NaN,52.0,764.0,hd,true,What Do These Creepy Plant Mouths Do? (Plant Tissues),Botany,4.0,54,False,regular course,7


### Add day of the week published 

In [119]:
from datetime import datetime

In [120]:
cc_bycourse_df["publishedAt"] = cc_bycourse_df["publishedAt"].apply(lambda x: datetime.fromisoformat(x[:-1]))


In [121]:
cc_bycourse_df["publishedDate"] = cc_bycourse_df["publishedAt"].apply(lambda x: x.date())
cc_bycourse_df["publishedTime"] = cc_bycourse_df["publishedAt"].apply(lambda x: x.time())
cc_bycourse_df.drop(columns=["publishedAt"], inplace=True)
cc_bycourse_df.head(2)

,video_id,channelTitle,description,tags,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,episode_title,course,episode_num,episode_title_len,with_Arizona,video_type,tag_count,publishedDate,publishedTime
0,WzOrF5W4l3Q,CrashCourse,"Plants and trees may seem pretty passive, but behind the scenes, their cells are working hard to...","[vlogbrothers, Crash Course, crashcourse, education, botany, photosynthesis, mitochondria, chlor...",13760.0,835.0,NaN,51.0,780.0,hd,true,Photosynthesis and Cellular Respiration,Botany,5.0,40,False,regular course,15,2023-06-22,16:00:21
1,pqosCR6J2qo,CrashCourse,"Plants—they’re just like us! Well, not exactly, but they do have skin and hair like us…even if t...","[vlogbrothers, Crash Course, crashcourse, education, botany, photosynthesis, plant fibers]",32496.0,1570.0,NaN,52.0,764.0,hd,true,What Do These Creepy Plant Mouths Do? (Plant Tissues),Botany,4.0,54,False,regular course,7,2023-06-15,16:00:38


### Include time since published 

In [122]:
cc_bycourse_df["days_published_for"] = cc_bycourse_df["publishedDate"].apply(lambda x: (datetime.today().date() - x).days)
cc_bycourse_df["days_published_for"]

0          1
1          8
2         10
3         17
4         22
        ... 
1470    4155
1471    4159
1472    4162
1473    4166
1474    4221
Name: days_published_for, Length: 1475, dtype: int64

### reorder columns

In [123]:
cc_bycourse_df.columns

Index(['video_id', 'channelTitle', 'description', 'tags', 'viewCount',
       'likeCount', 'favouriteCount', 'commentCount', 'duration', 'definition',
       'caption', 'episode_title', 'course', 'episode_num',
       'episode_title_len', 'with_Arizona', 'video_type', 'tag_count',
       'publishedDate', 'publishedTime', 'days_published_for'],
      dtype='object')

In [124]:
df_final = cc_bycourse_df[['video_id', 'channelTitle', 'course', "video_type", 'episode_title','episode_num', 'episode_title_len', 'tag_count', 'publishedDate', 'publishedTime', 'days_published_for', 'viewCount',
       'likeCount', 'favouriteCount', 'commentCount', 'duration', 'definition',
       'caption', "with_Arizona"]]
df_final

,video_id,channelTitle,course,video_type,episode_title,episode_num,episode_title_len,tag_count,publishedDate,publishedTime,days_published_for,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,with_Arizona
0,WzOrF5W4l3Q,CrashCourse,Botany,regular course,Photosynthesis and Cellular Respiration,5.0,40,15,2023-06-22,16:00:21,1,13760.0,835.0,NaN,51.0,780.0,hd,true,False
1,pqosCR6J2qo,CrashCourse,Botany,regular course,What Do These Creepy Plant Mouths Do? (Plant Tissues),4.0,54,7,2023-06-15,16:00:38,8,32496.0,1570.0,NaN,52.0,764.0,hd,true,False
2,xOLcZMw0hd4,CrashCourse,Biology,regular course,The Scientific Method,2.0,22,11,2023-06-13,16:00:44,10,30368.0,1519.0,NaN,87.0,909.0,hd,true,False
3,tZE_fQFK8EY,CrashCourse,Biology,regular course,Introduction to Biology,1.0,24,9,2023-06-06,16:00:39,17,70777.0,3775.0,NaN,266.0,807.0,hd,true,False
4,y9BLCfcUcFg,CrashCourse,Botany,regular course,Plant Cells & Hormones,3.0,23,22,2023-06-01,16:03:13,22,56483.0,3462.0,NaN,135.0,778.0,hd,true,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,HVT3Y3_gHGg,CrashCourse,Biology,regular course,Water - Liquid Awesome,2.0,23,26,2012-02-06,22:10:10,4155,4840139.0,51024.0,NaN,3825.0,677.0,hd,true,False
1471,n7ndRwqJYDM,CrashCourse,World History,regular course,Indus Valley Civilization,2.0,26,18,2012-02-02,20:18:11,4159,7917297.0,85507.0,NaN,8047.0,575.0,hd,true,False
1472,QnQe0xW_JY4,CrashCourse,Biology,regular course,Carbon... SO SIMPLE,1.0,20,18,2012-01-30,18:53:06,4162,7827237.0,78209.0,NaN,7809.0,717.0,hd,true,False
1473,Yocja_N5s1I,CrashCourse,World History,regular course,The Agricultural Revolution,1.0,28,27,2012-01-26,20:12:01,4166,15378590.0,152936.0,NaN,12247.0,671.0,hd,true,False


# Load as CSV

In [125]:
df_final.to_csv("data/crashcourse_bycourse_cleaned.csv")